# Code Coverage
In questo notebook aggiungiamo la code coverage dopo l'esecuzione dei nostri test.

Iniziamo con l'installare le librerie di cui abbiamo bisogno: Jacoco e Junit.

In [1]:
%maven org.jacoco:org.jacoco.core:0.8.7
    
%maven junit:junit:4.13.2
%maven org.junit.jupiter:junit-jupiter-engine:5.8.1
%maven org.junit.platform:junit-platform-launcher:1.8.1

## Code
Creiamo ila classe `Factorial` che sarà poi testato

In [18]:
public class Factorial{
    public static int factorial( int n ){
            int f = 1;

            for(int i = 1; i <=n; i++){
                f = f*i;
            }
            return f;
    }
}

## Junit
Creiamo un oggetto junit core e una funzione per printare il report dei nostri test

In [7]:
import static org.junit.Assert.*;
import org.junit.After;
import org.junit.Before;
import org.junit.Test;
import org.junit.runner.JUnitCore;
import org.junit.runner.Result;
import org.junit.runner.notification.Failure;

import java.util.*;


JUnitCore junit = new JUnitCore();

public static void resultReport(Result result) {
    System.out.println("Finished. Result: Failures: " +
      result.getFailureCount() + ". Ignored: " +
      result.getIgnoreCount() + ". Tests run: " +
      result.getRunCount() + ". Time: " +
      result.getRunTime() + "ms.");
    for (Failure f : result.getFailures()){
        System.out.println(f. getMessage());
        System.out.println(f. getTrace());
    }   
}

Creiamo la classe `FactorialTestCase` che testa la nostra classe `Factorial`

In [16]:
public class FactorialTestCase {
    @Test
    public void test_factorial_base() {
        Factorial f = new Factorial();
        int x = f.factorial(2);
        
        assertEquals(2, x);
    }
}

Eseguiamo i test normalmente per assicurarci che tutto stia funzionando correttamente

In [17]:
Result tmp_result = junit.run(FactorialTestCase.class);
resultReport(tmp_result);

Finished. Result: Failures: 0. Ignored: 0. Tests run: 1. Time: 0ms.


## Jacoco
Iniziamo la configurazione di jacoco

In [12]:
import java.io.InputStream;
import java.io.PrintStream;
import java.util.HashMap;
import java.util.Map;

import org.jacoco.core.analysis.Analyzer;
import org.jacoco.core.analysis.CoverageBuilder;
import org.jacoco.core.analysis.IClassCoverage;
import org.jacoco.core.analysis.ICounter;
import org.jacoco.core.data.ExecutionDataStore;
import org.jacoco.core.data.SessionInfoStore;
import org.jacoco.core.instr.Instrumenter;
import org.jacoco.core.runtime.IRuntime;
import org.jacoco.core.runtime.LoggerRuntime;
import org.jacoco.core.runtime.RuntimeData;

In [108]:
public class JacocoJupyterExecutor {
    JUnitCore junit;
    
    public JacocoJupyterExecutor(){
        junit = new JUnitCore();
    }
    
    public void execute(Class<?> objClass, Class<?> testClass) throws Exception {
        String targetName = objClass.getName();
        IRuntime runtime = new LoggerRuntime();
        Instrumenter instr = new Instrumenter(runtime);
        InputStream original = getTargetClass(targetName);
        byte[] instrumented = instr.instrument(original, targetName);
        original.close();

        RuntimeData data = new RuntimeData();
        runtime.startup(data);
        
        // JUNIT Test Exec
        Result tmp_result = junit.run(testClass);

        ExecutionDataStore executionData = new ExecutionDataStore();
        SessionInfoStore sessionInfos = new SessionInfoStore();
        data.collect(executionData, sessionInfos, false);
        runtime.shutdown();

        CoverageBuilder coverageBuilder = new CoverageBuilder();
        Analyzer analyzer = new Analyzer(executionData, coverageBuilder);
        original = getTargetClass(targetName);
        analyzer.analyzeClass(original, targetName);
        original.close();

        for (final IClassCoverage cc : coverageBuilder.getClasses()) {
            System.out.printf("class name:   %s%n", cc.getName());
            System.out.printf("class id:     %016x%n", Long.valueOf(cc.getId()));
            System.out.printf("\tinstructions: %s%n", Integer.valueOf(cc.getInstructionCounter().getTotalCount()));
            System.out.printf("\tbranches:     %s%n", Integer.valueOf(cc.getBranchCounter().getTotalCount()));
            System.out.printf("\tlines:        %s%n", Integer.valueOf(cc.getLineCounter().getTotalCount()));
            System.out.printf("\tmethods:      %s%n", Integer.valueOf(cc.getMethodCounter().getTotalCount()));
            System.out.printf("\tcomplexity:   %s%n%n", Integer.valueOf(cc.getComplexityCounter().getTotalCount()));
            for (int i = cc.getFirstLine(); i <= cc.getLastLine(); i++) {
                System.out.printf("Line %s: %s%n", Integer.valueOf(i),
                        getColor(cc.getLine(i).getStatus()));
            }
        }
    }

    private InputStream getTargetClass(final String name) {
        final String resource = '/' + name.replace('.', '/') + ".class";
        return getClass().getResourceAsStream(resource);
    }
    
    private String getColor(final int status) {
        switch (status) {
        case ICounter.NOT_COVERED:
            return "red";
        case ICounter.PARTLY_COVERED:
            return "yellow";
        case ICounter.FULLY_COVERED:
            return "green";
        }
        return "";
    }

}

In [109]:
JacocoJupyterExecutor jacocoRunner = new JacocoJupyterExecutor();

In [110]:
jacocoRunner.execute(Factorial.class, FactorialTestCase.class)

class name:   REPL/$JShell$52$Factorial
class id:     ace81003562f93ee
	instructions: 18
	branches:     2
	lines:        5
	methods:      2
	complexity:   3

Line 37: red
Line 38: 
Line 39: red
Line 40: 
Line 41: red
Line 42: red
Line 43: 
Line 44: red
